<a href="https://colab.research.google.com/github/Eran707/MSc-Computational-Neuroscience-Repo/blob/master/Multicompartment_Sim_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Neural Multicompartment Simulator**

**Eran Frank Shorer**

**MSc Computational Neuroscience**

**"Investigating the effects of impermeant anions on the electrical and computational properties of neurons"**




## Instructions:

This jupyter notebook consists of cells. Some cells are plain text, while others are code. 

Press Shift-Enter simultaneously, or click the cell and press the play button in the main toolbar above to run the cell. 

The output of the cell should be displayed below the cell. 



## **Import classes and modules** 


In [52]:
from ipywidgets import widgets, Layout, interact, interactive, interactive_output
from IPython.display import display
import common
import constants
import compartment
import electrodiffusion
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
print("All relevant classes imported")


All relevant classes imported


## **Set simulation parameters**

## *A) Compartment parameters*

### A1) Set compartment parameters:

In [56]:
txt_name = widgets.Text(value="Comp1")
lbl_name = widgets.Label(value ="Compartment Name: ")
name_box = widgets.HBox(children = [lbl_name,txt_name])

lbl_rad = widgets.Label(value = "Compartment Radius (um): ")
sldr_rad = widgets.FloatSlider(value=5, min=1, max=11.0, step=0.5, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
rad_box = widgets.HBox(children = [lbl_rad,sldr_rad])

lbl_len = widgets.Label(value = "Compartment Length (um): ")
sldr_len = widgets.FloatSlider(value=100, min=50, max=200, step=20, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
len_box = widgets.HBox(children = [lbl_len,sldr_len])

lbl_nai = widgets.Label(value = "Na+ concentration (mM): ")
sldr_nai = widgets.FloatSlider(value=14, min=0, max=100, step=10, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
hbox_nai = widgets.HBox(children = [lbl_nai,sldr_nai])

lbl_ki = widgets.Label(value = "K+ concentration (mM): ")
sldr_ki = widgets.FloatSlider(value=122, min=0, max=250, step=5, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
hbox_ki = widgets.HBox(children = [lbl_ki,sldr_ki])

lbl_cli = widgets.Label(value = "Cl- concentration (mM): ")
sldr_cli = widgets.FloatSlider(value=5, min=0, max=50, step=5, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
hbox_cli = widgets.HBox(children = [lbl_cli,sldr_cli])

lbl_xi = widgets.Label(value = "Impermeant anion concentration (mM): ")
sldr_xi = widgets.FloatSlider(value=150, min=0, max=300, step=10, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
hbox_xi = widgets.HBox(children = [lbl_xi,sldr_xi])

lbl_zi = widgets.Label(value = "Impermeant anion average charge: ")
sldr_zi = widgets.FloatSlider(value=-0.85, min=-10, max=10, step=0.5, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
hbox_zi = widgets.HBox(children = [lbl_zi,sldr_zi])



global name, comp_arr
name = txt_name.value
comp_arr =[]

output = widgets.Output()


btn_new_comp = widgets.Button(description = "Create Compartment",button_style ="success")


def new_comp_clicked(b):
    
    name = txt_name.value
    new_comp = compartment.Compartment(name,radius=sldr_rad.value*1e-5, length=sldr_len.value*1e-5)
    new_comp.set_ion_properties(na_i=sldr_nai.value*1e-3, k_i = sldr_ki.value*1e-3, cl_i=sldr_cli.value*1e-3,x_i=sldr_xi.value*1e-3, z_i=sldr_zi.value)
    comp_arr.append(new_comp)
    with output:
        print("Compartment: ("+ new_comp.name +") created" )

      
btn_new_comp.on_click(new_comp_clicked)


vbox_grand = widgets.VBox(children = [name_box,rad_box,len_box, hbox_nai,hbox_ki,hbox_cli,hbox_xi,hbox_zi,btn_new_comp], layout=Layout(border ="solid"))

display(output,vbox_grand)







Output()

In [58]:
x = 0
for i in comp_arr:
    print(comp_arr[x].name + ": " + str(comp_arr[x].na_i)+ " Molar")
    x += 1

Comp1: 0.06 Molar
Comp2: 0.06 Molar
Comp3: 0.06 Molar


### A2) Review compartments

In [59]:
tabs =[]
children =[]
aa = 0
m =0
for a in comp_arr:
    tabs.append(comp_arr[aa].name)
    lbl_name = widgets.Label(value = "Name")
    txt_name = widgets.Text(value = comp_arr[aa].name)
    hbox_name = widgets.HBox(children =[lbl_name,txt_name])
    lbl_rad = widgets.Label(value = "Radius")
    txt_rad = widgets.Text(value = str(comp_arr[aa].radius))
    hbox_rad = widgets.HBox(children =[lbl_rad,txt_rad])
    lbl_len = widgets.Label(value = "Length")
    txt_len = widgets.Text(value = str(comp_arr[aa].length))
    hbox_len = widgets.HBox(children =[lbl_len,txt_len])
    lbl_nai = widgets.Label(value = "Na+ mM")
    txt_nai = widgets.Text(value = str(comp_arr[aa].na_i))
    hbox_nai = widgets.HBox(children =[lbl_nai,txt_nai])
    lbl_ki = widgets.Label(value = "K+ mM")
    txt_ki = widgets.Text(value = str(comp_arr[aa].k_i))
    hbox_ki = widgets.HBox(children =[lbl_ki,txt_ki])
    lbl_cli = widgets.Label(value = "Cl- mM")
    txt_cli = widgets.Text(value = str(comp_arr[aa].cl_i))
    hbox_cli = widgets.HBox(children =[lbl_cli,txt_cli])
    lbl_xi = widgets.Label(value = "Impermeants mM")
    txt_xi = widgets.Text(value = str(comp_arr[aa].x_i))
    hbox_xi = widgets.HBox(children =[lbl_xi,txt_xi])
    lbl_zi = widgets.Label(value = "Impermeant charge")
    txt_zi = widgets.Text(value = str(comp_arr[aa].z_i))
    hbox_zi = widgets.HBox(children =[lbl_zi,txt_zi])
    vbox_comp = widgets.VBox(children =[hbox_name, hbox_rad,hbox_len,hbox_nai,hbox_ki,hbox_cli,hbox_xi,hbox_zi])
    children.append(vbox_comp)
    aa +=1

    
    
comp_tab = widgets.Tab()
comp_tab.children = children

bb= 0
for b in comp_arr:
    comp_tab.set_title(bb,tabs[bb])
    bb +=1

display(comp_tab)

#tab_contents = ['C0', 'C1', 'C2', 'C3', 'C4']
#children = [widgets.Text(description=name) for name in tab_contents]
#tab = widgets.Tab()
#tab.children = children
#for i in range(len(children)):
    #tab.set_title(i, str(i))
#tab""""""

## *B) Timing*

In [60]:
lbl_total_t = widgets.Label(value="Set the simulation run time (minutes):")
sldr_total_t = widgets.FloatSlider(value=3, min=1, max=20, step=0.5, disabled=False, continuous_update=False, orientation='horizontal',
    readout=True, readout_format='.1f')
HBox_total_t = widgets.HBox(children = [lbl_total_t,sldr_total_t])

lbl_dt = widgets.Label(value="Set the simulation time step (milliseconds):")
sldr_dt = widgets.FloatSlider(value=1, min=1, max=10, step=0.5, disabled=False, continuous_update=False, orientation='horizontal',
    readout=True, readout_format='.1f')
HBox_dt = widgets.HBox(children = [lbl_dt,sldr_dt])

dt= sldr_dt.value*1e-3 # 1ms time steps 
total_t= sldr_total_t.value *60 #s

run_t = 0 #current simulation timing
t_arr = [0]


widgets.VBox([HBox_total_t,HBox_dt], layout=Layout(border='solid'))


#print("Set total simulation time (minutes):")

#print("Set time step (milliseconds):")


In [19]:
comp_1 = compartment.Compartment("comp_1")
comp_1.set_ion_properties()
comp_2 = compartment.Compartment("comp_2")
comp_2.set_ion_properties()
ed_1_2 = electrodiffusion.Electrodiffusion(comp_1,comp_2)

## **Run Simulation**

In [13]:
while run_t < total_t:

    
    # will need to loop through comp_arr to update all these values
    
    
    comp_1.step(dt)
    comp_2.step(dt)
    comp_1_ed_dict = comp_1.get_ed_dict()
    comp_2_ed_dict = comp_2.get_ed_dict()
    ed_conc_changes = ed_1_2.calc_ed(dt,comp_1_ed_dict,comp_2_ed_dict)
    comp_1.ed_update(ed_conc_changes)
    for j in ed_conc_changes:
        ed_conc_changes[j] *= -1
    comp_2.ed_update(ed_conc_changes)
    comp_1.update_volumes()
    comp_1.update_arrays()
    run_t += dt
    t_arr.append(run_t)



TypeError: 'dict_keys' object is not subscriptable